In [1]:
#Import packages.
import pandas as pd
import json

In [2]:
def extract_ann(filename, column_name, category_num):
    if ((not isinstance(category_num, list)) | (not isinstance(column_name, list))):
        print("Your category is not a list")
    else:
        #Open the faces json.
        with open(filename) as json_data:
            data = json.load(json_data)
        df = pd.DataFrame(data['images'])
        ann = pd.DataFrame(data['annotations'])
    
        for category, name in zip(category_num, column_name):
            data_sub = ann[ann['category_id']==category]
            sub_set = set(data_sub['image_id'].to_list())
            df[name] = df['id'].isin(sub_set).astype(int)
    return df
    

In [11]:
data1 = extract_ann("./cocoface_instances_train2017.json", ['people', 'faces'], [1,2])
data2 = extract_ann("./instances_train2017.json", ['people_coco'], [1])
data3 = extract_ann("./cocoface_instances_val2017.json", ['people','faces'], [1,2])
data4 = extract_ann("./instances_val2017.json", ['people_coco'], [1])

In [14]:
data_faces = pd.concat([data1, data3])
data_coco = pd.concat([data2, data4])
data_all = data_coco.merge(data_faces, how='outer', on=['id', 'file_name', 'coco_url'])
data_all = data_all[['id', 'file_name', 'coco_url', 'people_coco', 'people', 'faces']]

#Replace nans with 0s.
data_all.fillna(0, inplace=True)

In [16]:
#Save to csv.
data_all.to_csv("./coco_annotations_socnosoc.csv")